In [ ]:
# print(f"null max:,{loan_df_dropnull.isnull().sum(axis=1).max()}")

In [ ]:
# loan_df_dropnull = loan_df[loan_df.isnull().sum(axis=1) < loan_df.isnull().sum(axis=1).max()]
# loan_df_dropnull

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.experimental import enable_iterative_imputer  # Required for IterativeImputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import fetch_openml

# ❌ Wrong pipeline

In [ ]:
encode_pipeline = Pipeline(
    steps=[
        ("binary_cols" , OrdinalEncoder()),
        ("ordinal_cols" ,OrdinalEncoder(categories=[['0', '1', '2', '3+']])),
        ("nominal_cols" ,OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ]
)

impute_pipeline = Pipeline(
    steps=[
        ("binary_cols" ,IterativeImputer()),
        ("ordinal_cols" ,IterativeImputer()),
        ("nominal_cols" ,IterativeImputer())
    ]
)

numerical_scale_pipeline = Pipeline(
    steps = [
        ("Scaler",StandardScaler())
        ]
)

# ❌ Wrong use of columntransformer

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

StandardScaler_numerical = ColumnTransformer(
    transformers=[("scale",StandardScaler(),numerical_cols)]
)

MinMaxScaler_numerical = ColumnTransformer(
    transformers=[("scale",MinMaxScaler(),numerical_cols)]
)

In [ ]:
log_reg_clf_pipeline = Pipeline(
    steps=[
        ("col_trans" ,col_trans),
        ("scale",StandardScaler_numerical),
        ("model", log_reg_clf)
    ]
)


col_trans is presumably a ColumnTransformer that already handles encoding (binary, ordinal, nominal). ✅

Then you’re doing another StandardScaler step (StandardScaler_numerical) which itself is a ColumnTransformer.

Here’s the issue:

After col_trans, the output is a NumPy array, because ColumnTransformer returns an array by default.

StandardScaler_numerical is a ColumnTransformer that tries to select columns by names (numerical_cols), but the array no longer has column names → ValueError.